In [ ]:
import schedule
import time
from binance.client import Client
from binance.exceptions import BinanceAPIException
import pymysql.cursors

# Connect to the database
db = pymysql.connect(
    host='usa.mysql.database.azure.com',
    user='dbusa',
    password='ahsan@123',
    db='python_db',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor,
    ssl={'ca': 'DigiCertGlobalRootCA.crt.pem'}
)

# Binance API initialization
api_key = 'vijG9ARQVjevGky2bkTgFACUMmRJBVxVqwtzDIONOToNIw4YmTMxMgyy4PEVUhFH'
api_secret = 'f9BHEpkASNW9S0nd7xe1Iq55ccd0bFrwI8LL0mA0UlIgODhubsX8noG6TnWicvmh'
client = Client(api_key, api_secret)

cursor = db.cursor()

# List of coins
coins = ["sxp", "chess", "blz", "joe", "perl", "ach", "gmt", "xrp", "akro", "zil"]

# Create table for each coin if it does not exist
for coin in coins:
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {coin}usdt (
            id INT AUTO_INCREMENT PRIMARY KEY,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            price DECIMAL(18, 8),
            volume DECIMAL(18, 8)
        )
    """)

def fetch_recent_trades(coin):
    try:
        trades = client.get_recent_trades(symbol=f"{coin.upper()}USDT")
        if trades:
            for trade in trades:
                price = trade['price']
                volume = trade['qty']
                # Insert data into MySQL
                cursor.execute(f"INSERT INTO {coin}usdt (price, volume) VALUES (%s, %s)", (price, volume))
            db.commit()
    except BinanceAPIException as e:
        print(f"An error occurred while fetching data for {coin.upper()}USDT: {str(e)}")
    except pymysql.Error as err:
        print(f"Error occurred: {err}")
        db.rollback()  # Revert the changes if an error occurred

def job():
    for coin in coins:
        fetch_recent_trades(coin)

# Schedule the job to run every 30 seconds
schedule.every(10).seconds.do(job)

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(5)
